### Mesh processing for SimuCell3D

This notebook outlines how to generate a geometry for the SimuCell3D.

In [1]:
import os
import sys
import numpy as np
from MeshPrep import isolate_filtered_meshes
from MeshPrep import mesh_process_clean, string_to_array

In [2]:
sys.path.append('/nas/groups/iber/Users/Federico_Carrara/Statistics_Collection/EpiStats/src/statistics_collection/')
from StatsAnalytics import prepare_df

#### Cleaning the labels

The labels generated from manual curation should be processesed before passing through the geometry generation. Strongly recommended step. Generates meshes which may be used for manual cell patch selection using paraview.

In [3]:
root = '../outputs/outputs_v3/output_intestine_sample2_b_curated_segmentation_relabel_seq_s_10_e_6_d_8/'

In [4]:
voxel_resolution = np.array([0.325, 0.325, 0.25])
label_path = os.path.join(root, 'processed_labels.tif')

In [ ]:
from VoxelProcessing import full_label_processing

output_folder='path/folder/to/save/processed/labels'
cell_info = full_label_processing(labeled_img=label_path, voxel_resolution=voxel_resolution, output_folder=output_folder, smoothing_iterations=10)
label_path = os.path.join(output_folder, "processed_labels.npy")
filtered_cell_list = cell_info[2]

#### Select labels for Simulations

To obtain labels for simulation one may use napari and select cells manually, or use meshes, and paraview. View how to use paraview [here](Tutorials/LabelSelection.md).

You may want to isolate the cells that do not touch the border instead. To isolate these use the following script and use Paraview's extract functionality as outlined.

In [5]:
cell_stats_df = prepare_df([
    os.path.join(root, 'cell_stats/stats_dataset_intestine_villus.csv')
])
filtered_row_idxs = np.nonzero(~cell_stats_df['exclude_cell'])[0]
filtered_cell_list = np.asarray(cell_stats_df['cell_ID'])[filtered_row_idxs]

In [7]:
mesh_path = os.path.join(root, 'cell_meshes')
# This is where the isolated meshes will be saved.
isolated_mesh_path = '../../Meshes_for_Simulation/examples/cell_clump_intestine' 

isolate_filtered_meshes(mesh_path, isolated_mesh_path, filtered_cell_list)

Converting files to .vtk: 100%|██████████| 239/239 [01:24<00:00,  2.83it/s]


Now you can open the `.vtk` mesh files in paraview and select a clump of cells for simulation.

#### Mesh Cleaning for SimuCell3D

Produce meshes for SimuCell3D simulation framework. Ensure that the path to labels is of cleaned, processed labels. 

In [8]:
label_list = string_to_array("182 183 196 206 207 225 229 233 235 242 249 251 254 257 260 262 264 268 273 274 282 313")
output_dir = '../../Meshes_for_Simulation/examples/cell_clump_intestine/cell_clumps/top_clump_22_cells/clean_meshes'

# Call the mesh_process_clean function
mesh_process_clean(
    label_path=label_path, 
    output_dir=output_dir, 
    label_list=label_list, 
    voxel_resolution=voxel_resolution, 
    scale_factor=1, 
    make_shell=True
)

-------------------------------------------
Creating meshes from labeled img...


Converting labels to meshes:  72%|███████▏  | 21/29 [04:41<01:45, 13.24s/it]